In [20]:
import re
import csv
import string
import operator

In [ ]:
import nltk
#nltk.download('all')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

Auxiliar functions
- remove_scenes_actions
- generate_ngrams
- build_dict

In [22]:
def remove_scenes_actions(line):
    new = ""
    found = False
    for letter in line:
        if letter == "(" or letter =="[":
            found = True
        if letter == ")" or letter =="]":
            found = False

        #outside brackets
        if found == False and letter!=")" and letter!="]":
            new = new + letter

    
    return(new)

In [24]:
def generate_ngrams(sentence: str, n: int):
    # Convert sentence to lowercase and remove punctuation
    import string
    sentence = sentence.lower().translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()

    # Generate N-grams
    ngrams = []
    for i in range(len(words) - n + 1):
        ngrams.append(' '.join(words[i:i+n]))

    return ngrams

In [80]:
def build_dict(data, characters,n):
    values = []

    for character in characters:
            lines = []

            for row in data:
                if row[-2] == character:
                    #we only want dialogue
                    str = remove_scenes_actions(row[-1])
                    lines.append(str)

            n_grams = []

            for line in lines:
                #remve punctuation
                line = line.translate(str.maketrans('', '', string.punctuation))

                n_gram = generate_ngrams(line,n)
                for word in n_gram:
                    n_grams.append(word)

            counter = {}
            for elem in n_grams:
                if elem in counter:
                    counter[elem]+=1
                else:
                    counter[elem]=1

            
            counter_sorted = sorted(counter.items(), key=operator.itemgetter(1))
            counts = counter_sorted[::-1]

            values.append(counts)

    dict = {key: value for key, value in zip(characters, values)}

    return dict

In [25]:
file = open("friends.csv", 'r', encoding="utf8")
csvreader = csv.reader(file)
data = list(csvreader)

characters =["Rachel", "Monica", "Joey", "Chandler", "Ross", "Phoebe"]

stop_words = set(stopwords.words('english'))

UNIGRAMS

In [61]:
values = []

for character in characters:
        lines = []
        counts = []
        words = []

        for row in data:
            if row[-2] == character:
                #we only want dialogue
                str = remove_scenes_actions(row[-1])
                lines.append(str)

        for line in lines:
            #remve punctuation
            line = line.translate(str.maketrans('', '', string.punctuation))
            #sp_character.append(line)
            
            line = word_tokenize(line)

            for word in line:
                if word not in stop_words and word != "’":
                    words.append(word.lower())

        counts = Counter(words)
        counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

        values.append(counts)

dict = {key: value for key, value in zip(characters, values)}

Transform the array *counts* to a dataframe and output it 

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(dict, orient='index')

# displaying the DataFrame
df.style


BIGRAMS

In [ ]:
dict = build_dict(data, characters, 2)

df = pd.DataFrame.from_dict(dict, orient='index')

# displaying the DataFrame
df.style

TRIGRAMS 

In [ ]:
dict = build_dict(data, characters, 3)

df = pd.DataFrame.from_dict(dict, orient='index')

# displaying the DataFrame
df.style

4-GRAMS

In [ ]:
dict = build_dict(data, characters, 4)

df = pd.DataFrame.from_dict(dict, orient='index')

# displaying the DataFrame
df.style

5-GRAMS

In [ ]:
dict = build_dict(data, characters, 5)

df = pd.DataFrame.from_dict(dict, orient='index')

# displaying the DataFrame
df.style

TF-IDF of the n-grams

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

for character in characters:
    lines = []
    sp_character = []
    
    for row in data:
        if row[-2] == character:
            #we only want dialogue
            str = remove_scenes_actions(row[-1])

            lines.append(str)

    ngram_range = (1, 4)

    # Define a custom tokenizer that only includes alphanumeric characters
    def tokenizer_new(text):
        # Convert to lowercase and split into words
        words = text.lower().split()
        # Remove punctuation and return only alphanumeric characters
        return [re.sub(r'\W+', '', w) for w in words]

    # Tokenize the texts into individual words and n-grams
    tokenizer = nltk.tokenize.word_tokenize
    tfidf = TfidfVectorizer(stop_words='english', tokenizer=tokenizer_new, ngram_range=ngram_range)

    # Compute the TF-IDF scores for all the n-grams in all the texts combined
    tfidf_scores = tfidf.fit_transform(lines).toarray()[0]

    # Get the n-gram vocabulary and their corresponding indices in the tfidf_scores array
    ngram_vocab = tfidf.get_feature_names_out()

    # Print the TF-IDF scores for each n-gram in descending order
    #for ngram, score in sorted(zip(ngram_vocab, tfidf_scores), key=lambda x: x[1], reverse=True):
    #    print("{}: {}".format(ngram, score))


    # Create a pandas DataFrame to store the n-grams and their corresponding TF-IDF scores
    df = pd.DataFrame({'ngram': ngram_vocab, 'tfidf': tfidf_scores})

    # Sort the DataFrame by TF-IDF score in descending order
    df = df.sort_values(by='tfidf', ascending=False)

    # Print the DataFrame
    print(character)
    print(df)